In [3]:
import os
import re
import sqlite3
from datetime import datetime
from typing import Optional, Dict, Any, List

import pandas as pd
import requests
from bs4 import BeautifulSoup

try:
    import pdfplumber
    HAS_PDFPLUMBER = True
except ImportError:
    HAS_PDFPLUMBER = False

BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, "data")
RAW_DIR = os.path.join(DATA_DIR, "raw")
os.makedirs(RAW_DIR, exist_ok=True)

print("HAS_PDFPLUMBER =", HAS_PDFPLUMBER)
print("DATA_DIR =", DATA_DIR)
print("RAW_DIR  =", RAW_DIR)



HAS_PDFPLUMBER = False
DATA_DIR = /Users/adnanaltimeemy/data
RAW_DIR  = /Users/adnanaltimeemy/data/raw


In [5]:
import re
import requests
from typing import Optional, Dict, Any, List

from bs4 import BeautifulSoup

try:
    import pdfplumber
    HAS_PDFPLUMBER = True
except ImportError:
    HAS_PDFPLUMBER = False


def fetch_url(url: str, timeout: int = 30) -> requests.Response:
    r = requests.get(
        url,
        timeout=timeout,
        headers={"User-Agent": "Mozilla/5.0"}
    )
    r.raise_for_status()
    return r


def save_binary(url: str, out_path: str) -> str:
    r = fetch_url(url)
    with open(out_path, "wb") as f:
        f.write(r.content)
    return out_path


def parse_money_to_number(text: str) -> Optional[float]:
    if not text:
        return None

    t = text.strip().lower().replace(",", "")
    m = re.search(r"([£€$])\s*([\d.]+)\s*(bn|b|m|k)?", t)
    if not m:
        return None

    _, num, suffix = m.groups()
    val = float(num)

    mult = 1.0
    if suffix in ("bn", "b"):
        mult = 1e9
    elif suffix == "m":
        mult = 1e6
    elif suffix == "k":
        mult = 1e3

    return val * mult


def soup_text(url: str) -> str:
    html = fetch_url(url).text
    soup = BeautifulSoup(html, "html.parser")

    for tag in soup(["script", "style", "noscript"]):
        tag.decompose()

    return " ".join(soup.get_text(" ").split())


def extract_metrics_from_press_text(text: str) -> Dict[str, Any]:
    out: Dict[str, Any] = {}

    m = re.search(
        r"revenue.*?\bto\b\s*(£\s*[\d.]+\s*(?:bn|m))",
        text,
        flags=re.IGNORECASE,
    )
    if m:
        out["revenue_reported"] = m.group(1)
        out["revenue_value"] = parse_money_to_number(m.group(1))

    m = re.search(
        r"underlying profit.*?\b(?:to|was)\b\s*(£\s*[\d.]+\s*(?:bn|m))",
        text,
        flags=re.IGNORECASE,
    )
    if m:
        out["underlying_profit_reported"] = m.group(1)
        out["underlying_profit_value"] = parse_money_to_number(m.group(1))

    m = re.search(
        r"(?:employs|employing|employed)\s+(?:around\s+)?([\d,]+)\s+people",
        text,
        flags=re.IGNORECASE,
    )
    if m:
        out["employees_reported"] = m.group(1)
        out["employees_value"] = int(m.group(1).replace(",", ""))

    m = re.search(
        r"(?:over|more than)\s+([\d.]+)\s+million\s+customers",
        text,
        flags=re.IGNORECASE,
    )
    if m:
        out["customers_millions_reported"] = m.group(1)
        out["customers_value"] = float(m.group(1)) * 1e6

    return out


def pdf_extract_text_first_pages(pdf_path: str, max_pages: int = 8) -> str:
    if not HAS_PDFPLUMBER:
        raise RuntimeError("pdfplumber is not installed. Run: pip install pdfplumber")

    text_parts: List[str] = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages[:max_pages]:
            page_text = page.extract_text()
            if page_text:
                text_parts.append(page_text)

    return "\n".join(text_parts)



In [6]:
SOURCES = [
    {
        "source_id": "press_2024_majority_niva_bupa",
        "url": "https://www.bupa.com/news-and-press/press-releases/2024/bupa-becomes-majority-shareholder-niva-bupa",
        "period_label": "FY2024 context",
        "doc_type": "press_release_html",
        "topic": "customers_employees"
    },
    {
        "source_id": "press_hy_2025_results",
        "url": "https://www.bupa.com/news-and-press/press-releases/2025/bupa-group-half-year-financial-results-2025",
        "period_label": "HY2025",
        "doc_type": "press_release_html",
        "topic": "financials_sales_profit"
    },
    {
        "source_id": "pdf_hy_2025_financial_statement",
        "url": "https://www.bupa.com/~/media/files/b/bupa-v5/documents/financials/results-centre/2025/bupa-group-results-and-financial-statement-hy2025.pdf",
        "period_label": "HY2025",
        "doc_type": "pdf",
        "topic": "financials_sales_profit_staff"
    },
    {
        "source_id": "pdf_annual_report_2024",
        "url": "https://www.bupa.com/~/media/files/b/bupa-v5/documents/annual-report-2024/bupa-group-2024-annual-report-and-accounts.pdf",
        "period_label": "FY2024",
        "doc_type": "pdf",
        "topic": "financials_sales_profit_staff"
    },
]

len(SOURCES)


4

In [8]:
from datetime import datetime, UTC
import os
import re
import pandas as pd

rows = []

for src in SOURCES:
    url = src["url"]
    source_id = src["source_id"]
    period = src["period_label"]
    doc_type = src["doc_type"]
    topic = src["topic"]

    record = {
        "entity": "Bupa",
        "period": period,
        "source_id": source_id,
        "source_url": url,
        "doc_type": doc_type,
        "topic": topic,
        "fetched_at_utc": datetime.now(UTC).isoformat(timespec="seconds"),

        "revenue_reported": None,
        "revenue_value": None,
        "underlying_profit_reported": None,
        "underlying_profit_value": None,

        "employees_reported": None,
        "employees_value": None,
        "customers_millions_reported": None,
        "customers_value": None,

        "staff_costs_reported": None,
        "staff_costs_value": None,
        "avg_staff_cost_per_employee": None,

        "notes": None,
    }

    try:
        if doc_type == "press_release_html":
            text = soup_text(url)
            metrics = extract_metrics_from_press_text(text)
            record.update(metrics)

            if re.search(r"insurance", text, flags=re.IGNORECASE):
                record["notes"] = "Contains insurance-related sales context (keyword match)."

        elif doc_type == "pdf":
            pdf_path = os.path.join(RAW_DIR, f"{source_id}.pdf")
            if not os.path.exists(pdf_path):
                save_binary(url, pdf_path)

            if HAS_PDFPLUMBER:
                staff = pdf_find_staff_costs(pdf_path)
                record.update(staff)
                record["notes"] = (record["notes"] or "") + " PDF downloaded; staff-cost heuristic applied."
            else:
                record["notes"] = (record["notes"] or "") + " PDF downloaded; install pdfplumber to parse text."

        if record["staff_costs_value"] is not None and record["employees_value"]:
            record["avg_staff_cost_per_employee"] = (
                record["staff_costs_value"] / record["employees_value"]
            )

    except Exception as e:
        record["notes"] = f"ERROR while processing source: {e}"

    rows.append(record)

kb = pd.DataFrame(rows)



In [9]:
out_csv = os.path.join(DATA_DIR, "bupa_kb.csv")
kb.to_csv(out_csv, index=False)

out_sqlite = os.path.join(DATA_DIR, "bupa_kb.sqlite")
conn = sqlite3.connect(out_sqlite)
kb.to_sql("bupa_kb", conn, if_exists="replace", index=False)
conn.close()

out_csv, out_sqlite


('/Users/adnanaltimeemy/data/bupa_kb.csv',
 '/Users/adnanaltimeemy/data/bupa_kb.sqlite')

In [10]:
kb[["period","source_id","revenue_reported","underlying_profit_reported","employees_reported","staff_costs_reported","notes"]]


,period,source_id,revenue_reported,underlying_profit_reported,employees_reported,staff_costs_reported,notes
0,FY2024 context,press_2024_majority_niva_bupa,None,None,"82,000",None,Contains insurance-related sales context (keyw...
1,HY2025,press_hy_2025_results,£244m,£244m,None,None,Contains insurance-related sales context (keyw...
2,HY2025,pdf_hy_2025_financial_statement,None,None,None,None,PDF downloaded; install pdfplumber to parse t...
3,FY2024,pdf_annual_report_2024,None,None,None,None,PDF downloaded; install pdfplumber to parse t...


In [11]:
kb[["period","revenue_value","underlying_profit_value","employees_value","staff_costs_value","avg_staff_cost_per_employee"]]


,period,revenue_value,underlying_profit_value,employees_value,staff_costs_value,avg_staff_cost_per_employee
0,FY2024 context,NaN,NaN,82000.0,None,None
1,HY2025,244000000.0,244000000.0,NaN,None,None
2,HY2025,NaN,NaN,NaN,None,None
3,FY2024,NaN,NaN,NaN,None,None


In [12]:
import sqlite3, pandas as pd
conn = sqlite3.connect("data/bupa_kb.sqlite")
pd.read_sql_query("SELECT period, revenue_value, underlying_profit_value FROM bupa_kb", conn)


,period,revenue_value,underlying_profit_value
0,FY2024 context,NaN,NaN
1,HY2025,244000000.0,244000000.0
2,HY2025,NaN,NaN
3,FY2024,NaN,NaN
